# Load Packages

In [1]:
using Roots

include("printmat.jl")

printmatDate

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

### From Chapter on Bonds 1

In [3]:
"""
Y:  scalar or K vector of interest rates
cf: scalar or K vector of cash flows
m: K vector of times of cash flows
"""
function BondPrice3(Y,cf,m)                #cf is a vector of all cash flows
    cdisc = cf./((Y.+1).^m)                #c/(1+y1)^t1 + c/(1+y2)^t2 + ...+ c/(1+y2)^m
    P     = sum(cdisc)                     #price
    return P
end

BondPrice3

# Bond Return after a Sudden Interest Rate Change

The calculations below assume that the yield curve is flat, but that it can shift up or down. This assumption helps us connect to the classical literature on duration hedging.

The initial values are indicated by subscript $_0$ and the values after the interest rate change by the subscript $_1$. It is assumed that the change is very sudden, so the time to cash flows are (virtually) the same before and after.

In [4]:
m  = 5
θ₀ = 0.02                   #initial interest rate
θ₁ = 0.03                   #interest rate after sudden change

cf = [zeros(4);1]           #cash flow, 5-period zero-coupon bond          
m = 1:5                     #time periods of the cash flows (could also do cf = 1, m = 5)

P₀ = BondPrice3(θ₀,cf,m)     #don't really need this for a zero-coupon bond, but it
P₁ = BondPrice3(θ₁,cf,m)     #allows us to easily change to more complicated setting
ΔP = P₁ - P₀
ΔPrel = ΔP/P₀

printlnPs("P₀, P₁, change, relative change (as θ changes from $θ₀ to $θ₁) ",[P₀ P₁ ΔP ΔPrel])

P₀, P₁, change, relative change (as θ changes from 0.02 to 0.03)      0.906     0.863    -0.043    -0.048


In [5]:
ytm = fzero(y->BondPrice3(y,cf,m)-P₀,[-0.1;0.1])   #testing ytm calculation
printlnPs("\nJust checking that θ₀ and ytm are the same: ",[θ₀ ytm])


Just checking that θ₀ and ytm are the same:      0.020     0.020


# Macaulay's Duration

$D^M = \sum_{k=1}^{K} m_{k}\frac{cf_{k}}{\left(  1+\theta\right)  ^{m_{k}}P}$

A first-order Taylor approximation gives

$\frac{\Delta P}{P} = -D^M \times \frac{\Delta \theta}{1+\theta}$ 

In [6]:
"""
P:   scalar, bond price
cf:  scalar or K vector of cash flows
m:   K vector of times of cash flows
ytm: scalar, yield to maturity
"""
function BondDuration(P,cf,m,ytm)
    cdisc   = cf.*m./((1+ytm).^(m.+1))      #c/(1+y)^2 + 2c/(1+y)^3 + ...
    D       = sum(cdisc)                    #Duration, row vector
    Da      = D/P                           #adjusted duration
    Dmac    = D*(1+ytm)/P                   #Macaulays duration
    return D,Da,Dmac
end

BondDuration

In [7]:
(D,Da,Dmac) = BondDuration(P₀,cf,m,ytm)

printlnPs("Macaulay's duration: ",Dmac)

Δθ           = θ₁ - θ₀
ΔPapprox_rel = -Dmac*Δθ/(1+θ₀)

printlnPs("Exact and approximate relative price change (according to Macaulay's duration): ",[ΔPrel ΔPapprox_rel])

Macaulay's duration:      5.000
Exact and approximate relative price change (according to Macaulay's duration):     -0.048    -0.049


# Hedging a Liability Stream

We are short one $L$ bond, which is worth $P_L$. To hedge that we buy $h$ units of bond $H$. The balance is put on a money market account $M$ to make the initial value of the portfolio zero ($V=0$)

$V=hP_{H}+M-P_{L}$

Over a short time interval, the change in the portfolio value is

$\Delta V=h\Delta P_{H}-\Delta P_{L}$, which can be approximated as

$\Delta V  =-hD_{H}^{M}P_{H}\times\frac{\Delta\theta_{H}}{1+\theta_{H}}+D_{L}^{M} P_{L}\times\frac{\Delta\theta_{L}}{1+\theta_{L}}$

In the cells below, we assume that the yield curve is flat, so $\theta_{H} = \theta_{L}$ both before and after the change.

In [8]:
cf = [150;250]                   #cash flow of liability stream (as positive numbers)
m  = [1;2]                       #times of cash flows   
θ₀ = 0.05                        #initial interest rate
θ₁ = 0.04                        #interest rate after sudden change
Δθ = θ₁ - θ₀

PL₀ = BondPrice3(θ₀,cf,m)      #price at initial interest rate, no face value  
PL₁ = BondPrice3(θ₁,cf,m)      #price at new interest rate

printlnPs("Value of liability at interest rates $θ₀ and $θ₁: ",[PL₀ PL₁])

(_,_,Dmac) = BondDuration(PL₀,cf,m,θ₀)
printlnPs("\nMacaulay's duration of the liability: ",Dmac)

Value of liability at interest rates 0.05 and 0.04:    369.615   375.370

Macaulay's duration of the liability:      1.613


## Naive Hedging

The "naive" hedging invests the same amount in the hedge bond as the value of the liability, that is, $hP_{H}=P_{L}$ so $h=P_{L}/P_{H}$. This gives

$\frac{\Delta V}{P_{L}}\approx\left(  D_{L}^{M}-D_{H}^{M}\right)  \times
\frac{\Delta\theta}{1+\theta}$

It also means that the money market account is not used ($M=0$).

In the example below, the hedge bond has a much shorter duration than the liability.

In [9]:
PH₀ = BondPrice3(θ₀,100,0.5)             #0.5 year bill is used as hedge bond
PH₁ = BondPrice3(θ₁,100,0.5)      

h = PL₀/PH₀

printlnPs("Hedge bond: zero coupon bond with m=0.5 and face value of 100: price and hedge ratio h: ",[PH₀ h])

printlnPs("\nMacaulay's duration of the liability and of the hedge bond: ",Dmac,0.5)

ΔV_exact = h*(PH₁-PH₀) - (PL₁-PL₀)
printlnPs("\n(relative) change of hedge portfolio value (exact): ",ΔV_exact/PL₀)

ΔVrel_approx = (Dmac - 0.5)*Δθ/(1+θ₀)
printlnPs("\n(relative) change of hedge portfolio value (approx): ",ΔVrel_approx)

Hedge bond: zero coupon bond with m=0.5 and face value of 100: price and hedge ratio h:     97.590     3.787

Macaulay's duration of the liability and of the hedge bond:      1.613     0.500

(relative) change of hedge portfolio value (exact):     -0.011

(relative) change of hedge portfolio value (approx):     -0.011


## Illustrating the Problem with the Naive Hedging

by plotting the value of the liability ($P_L$) and of the hedge bond position ($hP_H$) at different interest rates.

In [10]:
θ = 0:0.01:0.1

(PL,PH) = (fill(NaN,length(θ)),fill(NaN,length(θ)))
for i = 1:length(θ)
    PL[i] = BondPrice3(θ[i],cf,m)  
    PH[i] = BondPrice3(θ[i],100,0.5)
end

plot(θ,PL,label="liability")
plot!(θ,h*PH,label="h* hedge bond price")
title!("Naive hedging: the hedge position is not interest rate sensitive enough")
xlabel!("Interest rate")
ylabel!("Value of position")
annotate!(0.02,350,"The interest rates are\n $θ₀ (before) and $θ₁ (after)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 0.10 
 
 
 350 
 
 
 360 
 
 
 370 
 
 
 380 
 
 
 390 
 
 
 400 
 
 
 Naive hedging: the hedge position is not interest rate sensitive enough 
 
 
 Interest rate 
 
 
 Value of position 
 
 
 
 
 
 
 
 liability 
 
 
 
 h* hedge bond price 
 
 
 The interest rates are 
 
 
 0.05 (before) and 0.04 (after)

## Duration Matching

Choose a hedge bond with the same duration as the liability and invest equally much into each.

This is a special case of the naive hedging where $D_{L}^{M}=D_{H}$. Clearly, this gives $\frac{\Delta V}{P_{L}}\approx 0$.

In [11]:
PH₀ = BondPrice3(θ₀,100,Dmac)             
PH₁ = BondPrice3(θ₁,100,Dmac)      
h = PL₀/PH₀
printlnPs("A zero coupon bond with m=$(round(Dmac,2)) and face value of 100: price and hedge ratio h: ",[PH₀ h])

ΔV_exact = h*(PH₁-PH₀) - (PL₁-PL₀)
printlnPs("\n(relative) change of hedge portolio value (exact): ",ΔV_exact/PL₀)

printlnPs("\n(relative) change of hedge portolio value (approx): is constructed to be 0")

A zero coupon bond with m=1.61 and face value of 100: price and hedge ratio h:     92.430     3.999

(relative) change of hedge portolio value (exact):     -0.000

(relative) change of hedge portolio value (approx): is constructed to be 0


## Duration Hedging

In this case, $D_{L}^{M}\neq D_{H}$, but we adjust the hedge ratio $h$ to compensate for that. In particular, set 

$h =\frac{D_{L}^{M}}{D_{H}^{M}}\times\frac{P_{L}}{P_{H}}$ to get

$\frac{\Delta V}{P_{L}}\approx D_{L}^{M}\times\left(  \frac{\Delta\theta_{L} }{1+\theta_{L}}-\frac{\Delta\theta_{H}}{1+\theta_{H}}\right)$.

The balance $(P_L-hP_H)$ is kept on a money market account ($M$).

In [12]:
PH₀ = BondPrice3(θ₀,100,0.5)             
PH₁ = BondPrice3(θ₁,100,0.5)      
h = Dmac/0.5 * PL₀/PH₀
printlnPs("A zero coupon bond with m=0.5 and face value of 100: price and hedge ratio h: ",[PH₀ h])

ΔV_exact = h*(PH₁-PH₀) - (PL₁-PL₀)
printlnPs("\n(relative) change of hedge portolio value (exact): ",ΔV_exact/PL₀)

printlnPs("\n(relative) change of hedge portolio value (approx): is constructed to be 0")

printlnPs("\nNotice that we keep h*PH-PL on the money market account",PL₀-h*PH₀)

A zero coupon bond with m=0.5 and face value of 100: price and hedge ratio h:     97.590    12.222

(relative) change of hedge portolio value (exact):     -0.000

(relative) change of hedge portolio value (approx): is constructed to be 0

Notice that we keep h*PH-PL on the money market account  -823.129
